In [1]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime as dt
from random import randint
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 22
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
# get maximum lengths of the df
def get_len(worksheet, df):
    lengths = []
    for idx, col in enumerate(df):
        series = df[col]
        max_len = max((series.astype(str).map(len).max(),
                       len(str(series.name)))) + 2
        lengths.append(max_len)
    return lengths

In [3]:
# Main func of creating a pivot table
def create_report(path):
    xlsx_list = sorted(glob.glob(f'{path}*.xlsx'))
    output = pd.DataFrame()
    for name in xlsx_list:
        df = pd.read_excel(name, sheet_name = 'сводная таблица',
                           index_col = [0, 1],
                           usecols = [0, 1, 2, 3, 4])
        df.drop(index='Все', inplace=True)
        sums = df.groupby(level = 0).sum().drop(columns='Процент неликвидов')
        df.drop(columns = ['Всего новинок', 'Всего неликвидов'], inplace=True)
        df['Процент неликвидов'] /= 100
        sums['ИТОГ % НЕЛИКВИДОВ'] = sums['Всего неликвидов'] / sums['Всего новинок']
        sums.rename(columns={'Всего новинок':'ИТОГ ШТ. НОВИНОК',
                             'Всего неликвидов':'ИТОГ ШТ. НЕЛИКВИДОВ'}, inplace=True)
        sums['ИТОГ ШТ. НОВИНОК'] = sums['ИТОГ ШТ. НОВИНОК'].astype(str) + ' шт.'
        sums['ИТОГ ШТ. НЕЛИКВИДОВ'] = sums['ИТОГ ШТ. НЕЛИКВИДОВ'].astype(str) + ' шт.'
        sums = sums.stack().to_frame(name='Процент неликвидов')
        sums['dummy'] = 1
        period = pd.concat([df, sums])
        period.rename(columns={'Процент неликвидов':name[-15:-8]}, inplace=True)
        output = pd.concat([output, period], axis=1)
    output['sorter'] = output['dummy'].sum(axis=1)
    output = output.sort_values(by=['Редакция', 'sorter']).drop(columns=['dummy', 'sorter'])
    return output.reset_index()

In [4]:
# Write pivot table to the file
output = create_report('input/АСТ/')
# output = create_report('input/ЭКСМО/')
writer = pd.ExcelWriter(f'out.xlsx', engine='xlsxwriter') 
output.to_excel(writer, index=False, sheet_name='сводная таблица', na_rep='')

In [5]:
# Init objects
workbook = writer.book
max_row, max_col = output.shape
# editorials = ['100 - Редакция "Комильфо"',
#               '101 - Редакция № 1',
#               '102 - Редакция № 2',
#               '120 - Редакция № 5']
editorials = ['507 Департамент Планета Детства',
              '509 Департамент Художественной литературы',
              '510 Департамент Межиздат',
              '511 Департамент Прикладная литература']

In [6]:
# Main chart 
# Get chart data
chart1_data = output.loc[lambda df: df['Подразделение'].isin(['ИТОГ % НЕЛИКВИДОВ']), :]
chart1_data = chart1_data.loc[lambda df: df['Редакция'].isin(editorials), :]

# Add data to chart
chart1 = workbook.add_chart({'type': 'line'})
for i in chart1_data.index:
    chart1.add_series({
        'name': ['сводная таблица', i+1, 0, i+1, 0],
        'values': ['сводная таблица', i+1, 2, i+1, max_col-1],
        'marker': {'type': 'diamond', 'size': 5, 'fill': {'color': 'black'}},
        'categories' : ['сводная таблица', 0, 2, 0, max_col-1]})
    
# Set chart params
chart1.set_legend({'position':'bottom',
                   'font': {'size': 9}})
chart1.set_x_axis({'minor_gridlines': {'visible': True}})
chart1.set_y_axis({'minor_gridlines': {'visible': True}})

# Add chart to the list
worksheet = workbook.add_chartsheet('Редакции')
worksheet.set_chart(chart1)

In [7]:
# Specific charts
# Get chart data
for i, edtr in enumerate(editorials):
    if i == 0:
        continue
    elif i == 1:
        part1 = output.loc[lambda df: df['Редакция'].isin([editorials[1]]), :]
        part1.drop(part1.tail(3).index, inplace=True)
        part2 = output.loc[lambda df: df['Редакция'].isin([editorials[0]]), :]
        part2.drop(part2.tail(3).index, inplace=True)
        spec_chart_data = pd.concat([part1, part2])
    else:
        spec_chart_data = output.loc[lambda df: df['Редакция'].isin([edtr]), :]
        spec_chart_data.drop(spec_chart_data.tail(3).index, inplace=True)
        
# Add data to chart
    spec_chart = workbook.add_chart({'type': 'line'})
    for i in spec_chart_data.index:
        spec_chart.add_series({
            'name': ['сводная таблица', i+1, 1, i+1, 1],
            'values': ['сводная таблица', i+1, 2, i+1, max_col-1],
            'marker': {'type': 'diamond', 'size': 5, 'fill': {'color': 'black'}},
            'categories' : ['сводная таблица', 0, 2, 0, max_col-1]})
        
# Set chart params
    spec_chart.set_legend({'position':'bottom',
                           'font': {'size': 8}})
    spec_chart.set_x_axis({'minor_gridlines': {'visible': True}})
    spec_chart.set_y_axis({'minor_gridlines': {'visible': True}})
    worksheet = workbook.add_chartsheet(str(edtr)[:3])
    worksheet.set_chart(spec_chart)

In [8]:
# Init formats
bld_brdr = workbook.add_format({'bold': True,
                                'border': 1})
bld_brdr_clr = workbook.add_format({'bold': True,
                                    'border': 1,
                                    'bg_color': '#FABF8F',
                                    'num_format': '0.0%'})
bld_brdr_clr_l = workbook.add_format({'bold': True,
                                      'border': 1,
                                      'bg_color': '#FDE9D9',
                                      'num_format': '0.0%'})
perc = workbook.add_format({'num_format': '0.0%'})

In [9]:
# Format cells
worksheet = writer.sheets['сводная таблица']
lengths = get_len(worksheet, output)
itog_idx = output.loc[output['Подразделение'] == 'ИТОГ ШТ. НЕЛИКВИДОВ'].index
main_itog_idx = chart1_data.index

worksheet.set_column(2, max_col, 8, perc)
worksheet.set_column('A:A', lengths[0], bld_brdr)
worksheet.set_column('B:B', lengths[1], bld_brdr)

for idx in itog_idx:
    if idx+1 in main_itog_idx:
        clr = bld_brdr_clr
    else:
        clr = bld_brdr_clr_l
    worksheet.set_row(idx, None, clr)
    worksheet.set_row(idx+1, None, clr)
    worksheet.set_row(idx+2, None, clr)

0

0

0

0

0

...

0

0

0

0

0

In [10]:
writer.close()